#### CNN

여기서 C는 합성곱 전처리

이미지 분류 강점
---
전처리 과정 : padding >> convolution >> pooling >> flatten

In [12]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D 
from keras.callbacks import ModelCheckpoint,EarlyStopping
import matplotlib.pyplot as plt 
import numpy
import os
import tensorflow as tf

# seed 값 설정
seed = 0 
numpy.random.seed(seed) 
tf.random.set_seed(seed) # tf.set_random_seed(seed) 안됨

#### Fashion_Mnist CNN

In [ ]:
import pandas as pd

train_data = pd.read_csv('/content/fashion-mnist_train.csv')
test_data = pd.read_csv('/content/fashion-mnist_train.csv')

In [8]:
train_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0,0,0,0,0,0,0,5,0,...,0.0,0.0,0.0,30.0,43.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,1,2,0,0,0,0,...,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X = train_data.iloc[:,1:]
y = train_data.iloc[:,0]

X = np.array(X)
y = to_categorical(np.array(y))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state = 107)

In [ ]:
X_train.shape

In [ ]:
y_train[0]

In [ ]:
X_train[0]

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_val = X_val.reshape(X_val.shape[0], 28, 28, 1)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(X_train[1].reshape(28,28))
plt.show()

In [ ]:
# convolution(32, kernel_size=(3,3)) > pooling > drop out(0.2) >
# convolution(64) > pooling > drop out(0.2) >
# convolution(128) > drop out(0.2) > flatten
# dense 128 > drop > 출력층

# 컨볼루션 신경망 설정(전처리)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu')) 
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',
metrics=['accuracy'])

# 모델 최적화 설정
MODEL_DIR = './model2/'
if not os.path.exists(MODEL_DIR):
  os.mkdir(MODEL_DIR)

modelpath="./model2/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)
# 모델의 실행
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),\
                    epochs=30, batch_size=200, verbose=0, callbacks=[early_stopping_callback,checkpointer])

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test) [1])) # 테스트셋의 오차
y_vloss = history.history['val_loss'] # 학습셋의 오차
y_loss = history.history['loss']

# 그래프로 표현
x_len = numpy.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss') 
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

In [ ]:
# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right') 
plt.grid()
plt.xlabel('epoch') 
plt.ylabel('loss')
plt.show()